### libraries

In [1]:
import pandas as pd
import numpy as np
import csv

### Anime Tupe Rules

In [2]:
class AnimeRules:
  def __init__(self, dirty_dataset, cleaned_dataset):
    self.dirty_dataset = dirty_dataset
    self.cleaned_dataset = cleaned_dataset
    self.df_d = pd.read_csv(dirty_dataset)
    self.df_d['type'].fillna('',  inplace=True)
    
  def movie_rule(self):
        cnt = 0
        for index, row in self.df_d.iterrows():
            if (float(row['episodes']) == 1.0) and ("movie" not in row['type'] or row['type'] is ""):
                    cnt+=1
                            
        print("Number of Detected Tuples are {}".format(cnt))

  def series_rule(self):
        cnt = 0
        for index, row in self.df_d.iterrows():
            if (float(row['episodes']) != 1.0) and ("movie" in row['type'] or row['type'] is ""):
                    cnt+=1
                            
        print("Number of Detected Tuples are {}".format(cnt))
        
  def repairing(self):
        print(self.df_d['type'].count())
        for index, row in self.df_d.iterrows():
            if float(row['episodes']) != 1.0 and ("movie" in row['type'] or row['type'] is ""):
                    self.df_d.at[index,'type'] = "tv series"
            elif (float(row['episodes']) == 1.0) and ("movie" not in row['type'] or row['type'] is ""):
                    self.df_d.at[index,'type'] = "movie"
           
        print(self.df_d['type'].count())
        self.df_d.to_csv(self.cleaned_dataset, index = False)

### Error Detection & Repair

In [3]:
dirty_anime_data = "data/dirtyTrainDataset.csv"
cleaned_anime_data = "data/CleanedData.csv"

FixData = AnimeRules(dirty_anime_data, cleaned_anime_data)

print("Error detection of anime series")
FixData.series_rule()

print("Error detection of anime movies")
FixData.movie_rule()

print("repairing.....! check please data directory")
FixData.repairing()

Error detection of anime series
Number of Detected Tuples are 446
Error detection of anime movies
Number of Detected Tuples are 400
repairing.....! check please data directory
1600
1600


### Precision and Recall

In [4]:
original_data = pd.read_csv("data/TrainDataset.csv")
dirty_data = pd.read_csv("data/dirtyTrainDataset.csv")
cleaned_data = pd.read_csv("data/CleanedData.csv")

matrix = pd.DataFrame(columns=["predicted True", "predicted False"], index=["actual True", "actual False"], data=[[0,0],[0,0]])

print(original_data.head())
print(dirty_data.head())
print(cleaned_data.head())

for column in ["type"]:
    for i in range(0, len(cleaned_data[column])):
        
        # 1 - Cleaned in correct way 
        if cleaned_data[column][i] == original_data[column][i] and original_data[column][i] != dirty_data[column][i]:
            matrix["predicted True"]["actual True"] = matrix["predicted True"]["actual True"]+1
            #print("Detected error and fixed")
            
        # 2 - Cleaned in wrong way: Detected error and replaced wrong
        elif cleaned_data[column][i] != original_data[column][i] and original_data[column][i] != dirty_data[column][i]:
            matrix["predicted False"]["actual True"] = matrix["predicted False"]["actual True"]+1
            #print("Did not find error")
            
        # No cleaning necessary - no cleaning done
        elif cleaned_data[column][i] == original_data[column][i] and original_data[column][i] == dirty_data[column][i]:
            matrix["predicted False"]["actual False"] = matrix["predicted False"]["actual False"]+1
            #print("No error implemnted - nothing fixed")
            
        # Cleaned - no cleaning necessary
        elif cleaned_data[column][i] != original_data[column][i] and original_data[column][i] == dirty_data[column][i]:
            matrix["predicted True"]["actual False"] = matrix["predicted True"]["actual False"]+1
            #print("No error implemented - fixed wrong")
            
print("Detected error and fixed",matrix["predicted True"]["actual True"])
print("Did not find error",matrix["predicted False"]["actual True"])
print("No error implemnted - nothing fixed",matrix["predicted False"]["actual False"])
print("No error implemented - fixed wrong",matrix["predicted True"]["actual False"])



   anime_id                       name                       genre       type  \
0     12429    Anata ga Furikaeru Toki       Romance,School,Shoujo  tv series   
1     33888                    Jubilee  Comedy,Drama,Fantasy,Music      movie   
2      1860      Eiyuu Gaiden Mozaicka                   Animation      movie   
3      2848              Sky Girls OVA      Animation,Short,Sci-Fi      movie   
4     22625  Gakuen de Jikan yo Tomare                      Hentai      movie   

   episodes  rating  members  \
0         2    6.56     4621   
1         3    4.67       35   
2        14    7.31    43632   
3         1    7.65     6953   
4         1    6.00       75   

                                            overview  
0  'Kiteretsu Daihyakka is the story of Eichii Ki...  
1  'Queen Elisabeth I travels 400 years into the ...  
2                                                NaN  
3  'Earth is at war with aliens. The war has left...  
4  "In order to wish for immortality and aven